In [1]:
import torch
from chunkformer_vpb.training.finetune_utils import (
    get_default_args,
    prepare_input_file,
    load_model_only,
    compute_chunkformer_loss,
)
from chunkformer_vpb.training.tokenizer import normalize_vi, GreedyTokenizer

import os, math, torch
from chunkformer_vpb.training.finetune_config import FinetuneConfig
from chunkformer_vpb.training.data_loader     import get_dataloaders
from chunkformer_vpb.training.optimizer       import build_model_and_optimizer
from chunkformer_vpb.training.finetune_utils  import compute_loss_batch_v1, compute_loss_batch_v2


CFG_PATH   = "../../config/finetune_config.yaml"

def debug_text_pipeline(label_text: str, tokenizer: GreedyTokenizer):
    print(">>> Text pipeline debug <<<")
    print("  Original text :", label_text)
    norm = normalize_vi(label_text)
    print("  Normalized   :", norm)
    ids = tokenizer.tokenize(norm)
    print("  Token IDs    :", ids)
    try:
        dec = tokenizer.decode_ids(ids)
    except AttributeError:
        # nếu chưa có decode_ids
        dec = "".join(tokenizer.vocab[id] for id in ids)
    print("  Decoded      :", dec)
    print()

def main():
    # 1) chuẩn bị args + device
    args = get_default_args()
    args.model_checkpoint = "../../../chunkformer-large-vie"
    args.audio_path       = "./cache_train/raw/utt_000664.wav"
    args.label_text       = "một giọng nói du dương không thể lẫn với ai khác cất lên"

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # 2) load model + tokenizer
    model, _ = load_model_only(args.model_checkpoint, device)
    model.ctc_weight = 0.3
    tokenizer = GreedyTokenizer(vocab_path=f"{args.model_checkpoint}/vocab.txt")

    # 3) debug text pipeline (single)
    debug_text_pipeline(args.label_text, tokenizer)

    # 4) prepare single‐sample features + loss
    xs = prepare_input_file(args.audio_path, device)  # [1, T_raw, 80]
    print(">>> Single‐sample loss <<<")
    loss_s = compute_chunkformer_loss(
        model=model,
        tokenizer=tokenizer,
        xs=xs,
        args=args,
        label_text=args.label_text,
        device=device
    )
    print(f"Single: loss={loss_s['loss']:.3f},  ctc={loss_s['loss_ctc']:.3f}, att={loss_s['loss_att']:.3f}\n")

    # 5) now wrap into a batch of size 1 and run batch version
    #    (giả sử bạn có hàm compute_loss_batch_v1 cho batch)
    from chunkformer_vpb.training.finetune_utils import compute_loss_batch_v1
    feats = xs
    feat_lens = torch.tensor([xs.shape[1]], device=device)
    # build toks/tok_lens exactly như in DataLoader
    norm = normalize_vi(args.label_text)
    ids = tokenizer.tokenize(norm)
    toks = torch.LongTensor([ids]).to(device)
    tok_lens = torch.tensor([len(ids)], device=device)


    cfg = FinetuneConfig.from_yaml(CFG_PATH)

    # để nhanh: batch nhỏ & shuffle false
    cfg.training.batch_size = 1
    cfg.training.shuffle    = False

    print(">>> Batch‐of‐1 loss <<<")
    loss_b, loss_b_ctc, loss_b_att = compute_loss_batch_v1(
        model, feats, feat_lens, toks, tok_lens, cfg, device
    )
    print(f"Batch1: loss={loss_b:.3f},  ctc={loss_b_ctc:.3f}, att={loss_b_att:.3f}")

if __name__ == "__main__":
    main()


/home/kylh/.local/share/mamba/envs/stt310/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[2025-07-16 17:56:08] INFO: Checkpoint: loading from checkpoint ../../../chunkformer-large-vie/pytorch_model.bin for GPU



🧾 Loaded checkpoint from: ../../../chunkformer-large-vie/pytorch_model.bin
📦 Checkpoint keys: ['encoder.global_cmvn.mean', 'encoder.global_cmvn.istd', 'encoder.embed.out.weight', 'encoder.embed.out.bias', 'encoder.embed.conv.0.weight'] ... (total 813)
🔍 AED decoder head included in checkpoint? ✅ YES
📊 Model total params: 113,852,240, trainable: 113,852,240
>>> Text pipeline debug <<<
  Original text : một giọng nói du dương không thể lẫn với ai khác cất lên
  Normalized   : một giọng nói du dương không thể lẫn với ai khác cất lên
  Token IDs    : [4104, 2648, 4564, 2300, 2356, 3305, 5697, 3688, 6411, 1333, 3277, 2146, 3597]
  Decoded      : một giọng nói du dương không thể lẫn với ai khác cất lên

>>> Single‐sample loss <<<
>>>>>>>>>>>>>>>>>>>  ys_out shape: torch.Size([1, 14]), logp shape: torch.Size([1, 14, 6992])
Single: loss=0.430,  ctc=0.001, att=0.614

>>> Batch‐of‐1 loss <<<
[ENC_OFFLINE] start: B=1, T_raw_max=423, D=80
  [sample 0] raw_feat.shape = torch.Size([1, 423, 80]), ra